# Azure AI Search NVIDIA NIM ഉപയോഗിച്ച് LlamaIndex സംയോജനം

In this notebook, we'll demonstrate how to leverage NVIDIA's AI models and LlamaIndex to create a powerful Retrieval-Augmented Generation (RAG) pipeline. We'll use NVIDIA's LLMs and embeddings, integrate them with Azure AI Search as the vector store, and perform RAG to enhance search quality and efficiency.

## പ്രയോജനങ്ങൾ
- **Scalability**: NVIDIA 的 വില്പനാദായക Language മോഡലുകളും Azure AI Search ഉം ഉപയോഗിച്ച് സ്കേലബിൾയും കാര്യക്ഷമവുമായ റെട്രീവൽ നേടുക.
- **ചെലവ് കാര്യക്ഷമത**: കാര്യക്ഷമമായ വെക്ടർ സ്റ്റോറേജ് மற்றும் ഹൈബ്രിഡ് സെർച് സാങ്കേതികവിദ്യകൾ ഉപയോഗിച്ച് തിരയൽ மற்றும் റെട്രീവൽ ചെലവ് ഒപ്റ്റിമൈസ് ചെയ്യുക.
- **ഉയർന്ന പ്രകടനം**: ശക്‌തമാർന്ന LLMs നെ വെക്ടറൈസ്ഡ് സെർചുമായി സംയോജിപ്പിച്ച് വേഗതയും നിസ്സംശയതയും ലഭ്യമാക്കുക.
- **ഗുണമേന്മ**: ബന്ധപ്പെട്ട തിരിച്ചെടുത്ത ഡോക്യുമെന്റുകളാൽ LLM പ്രതികരണങ്ങളെ ഗ്രൗണ്ട് ചെയ്യത്തച്ചും ഉയർന്ന തിരയൽ ഗുണമേന്മ കாத்தുസൂക്ഷിക്കുക.

## മുൻ‌ആവശ്യങ്ങൾ
- 🐍 Python 3.9 അല്ലെങ്കിൽ അതിലധികം
- 🔗 [Azure AI Search സർവ്വീസ്](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API കീ — NVIDIA NIM മൈക്രോസർവീസുകൾ വഴി NVIDIA യുടെ LLMs અને Embeddings ആക്‌സസ് ചെയ്യുന്നതിനുള്ള

## ഉൾപ്പെടുത്തിയ സവിശേഷതകൾ
- ✅ NVIDIA LLM സംയോജനം (ഞങ്ങൾ [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) ഉപയോഗിക്കും)
- ✅ NVIDIA എംബെഡിംഗ്സ് (ഞങ്ങൾ [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) ഉപയോഗിക്കും)
- ✅ Azure AI Search ഉന്നത തിരയൽ മോഡുകൾ
- ✅ LlamaIndex ഉപയോഗിച്ച് ഡോക്യുമെന്റ് ഇൻഡെക്സിംഗ്
- ✅ NVIDIA LLMs ഉപയോഗിച്ച് Azure AI Search ഉം LlamaIndex ഉം ഉപയോഗിച്ച RAG

നമുക്ക് തുടങ്ങാം!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## ഇൻസ്റ്റലേഷൻവും ആവശ്യകതകളും
Python പതിപ്പ് >3.10 ഉപയോഗിച്ച് ഒരു Python പരിസ്ഥിതി സൃഷ്ടിക്കുക।

## തുടങ്ങാം!


പ്രാരംഭിക്കാൻ, NVIDIA AI Foundation മോഡലുകൾ ഉപയോഗിക്കാൻ നിങ്ങൾക്ക് `NVIDIA_API_KEY` ആവശ്യമാണ്:
1) [NVIDIA](https://build.nvidia.com/explore/discover) ൽ ഒരു സൗജന്യ അക്കൗണ്ട് സൃഷ്ടിക്കുക.
2) നിങ്ങളുടെ തിരഞ്ഞെടുക്കുന്ന മോഡലിൽ ക്ലിക്കുചെയ്യുക.
3) Input വിഭാഗത്തിൽ, Python ടാബ് തിരഞ്ഞെടുക്കുക, തുടർന്ന് **API കീ നേടുക** എന്നതിൽ ക്ലിക്കുചെയ്യുക, അതിനു ശേഷം **കീ സൃഷ്ടിക്കുക** എന്നതിൽ ക്ലിക്കുചെയ്യുക.
4) സൃഷ്ടിച്ച കീ നകൽ ചെയ്ത് NVIDIA_API_KEY എന്നായി സംരക്ഷിക്കുക. അതിനുശേഷം, നിങ്ങൾക്ക് എൻഡ്പോയിന്റുകൾക്ക് പ്രവേശനം ലഭിക്കണം.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## LLM, Embedding ഉപയോഗിച്ചുള്ള RAG ഉദാഹരണം
### 1) LLM ആരംഭിക്കുക
`llama-index-llms-nvidia`, NVIDIAയുടെ LLM കണക്ടറായിട്ടാണ് അറിയപ്പെടുന്നത്, ഇത് നിങ്ങള്‍ക്ക് NVIDIA API കാറ്റലോഗിലുള്ള അനുയോജ്യമായ മോഡലുകളുമായി കണക്ട് ചെയ്ത് അവയില് നിന്ന് ജനറേറ്റ് ചെയ്യാന് അനുവദിക്കുന്നു. ചാറ്റ് പൂർത്തീകരണ മോഡലുകളുടെ പട്ടികയ്‌ക്കായി ഇവിടെ കാണുക: https://build.nvidia.com/search?term=Text-to-Text

ഇവിടെ നാം사용ിക്കുന്നത് **mixtral-8x7b-instruct-v0.1** ആണ്


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) എംബെഡിംഗ് ആരംഭിക്കുക
`llama-index-embeddings-nvidia`, NVIDIAയുടെ Embeddings കണക്ടറെന്ന പേരിൽ അറിയപ്പെടുന്ന, NVIDIA API കാറ്റലോഗിൽ ലഭ്യമായ പൊരുത്തപ്പെടുന്ന മോഡലുകളുമായി നിങ്ങൾക്ക് കണക്ട് ചെയ്യാനും അവയിൽ നിന്ന് ജനറേറ്റ് ചെയ്യാനുമുള്ള സൗകര്യം നൽകുന്നു. നാം `nvidia/nv-embedqa-e5-v5` എന്നതിനെ എംബെഡിംഗ് മോഡലായി തിരഞ്ഞെടുത്തു. ടെക്സ്റ്റ് എംബെഡിംഗ് മോഡലുകളുടെ പട്ടികയ്ക്ക് ഇവിടെ കാണുക: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) ഒരു Azure AI Search വെക്റ്റർ സ്റ്റോർ സൃഷ്ടിക്കുക


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### 4) പ്രമാണങ്ങൾ ലോഡ് ചെയ്യുക, ഭാഗങ്ങളായി വിഭജിച്ച് അപ്‌ലോഡ് ചെയ്യുക


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) നിങ്ങളുടെ ഡാറ്റയിൽ ചോദ്യങ്ങൾ ചോദിക്കാൻ ഒരു ക്വറി എഞ്ചിൻ സൃഷ്ടിക്കുക

ഇതാ Azure AI Search-ൽ ശുദ്ധ വെക്ടർ തിരച്ചിൽ ഉപയോഗിച്ച് നടത്തിയ ഒരു ക്വറിയാണ്, അത് നമ്മുടെ LLM (Phi-3.5-MOE)-നിലേക്ക് പ്രതികരണം അടിസ്ഥാനമാക്കുന്നു


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

ഇത് Azure AI Search-ൽ ഹൈബ്രിഡ് സർച്ച് ഉപയോഗിച്ചുള്ള ഒരു ക്വറിയാണ്.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### വെക്ടർ തിരയൽ വിശകലനം
LLM പ്രതികരണം റഷ്യയുടെ ഷെയർ വിപണിയെക്കുറിച്ചുള്ള ഉറവിടത്തിൽ പരാമർശിച്ച പ്രധാന സാമ്പത്തിക പ്രത്യാഘാതങ്ങളെ കൃത്യമായി പിടിച്ചെടുത്തിരിക്കുന്നു. പ്രത്യേകിച്ച്, അത് പറയുന്നു: റഷ്യയുടെ ഷെയർ വിപണി വലിയൊരു ഇടിവ് അനുഭവിച്ചു, അതിന്റെ മൂല്യത്തിൽ 40% നഷ്ടം ഉണ്ടായി, നിലവിലുള്ള സാഹചര്യത്തെ തുടർന്ന് വ്യാപാരം താൽക്കാലികമായി നിർത്തിവെച്ചു. ഈ പ്രതികരണം ഉറവിടത്തിൽ നൽകിയ വിവരങ്ങളുമായി നന്നായി പൊരുത്തപ്പെടുന്നു; ഇത് LLM റഷ്യയുടെ നടപടികളുടെയും അതിന് വിധിച്ചിരിക്കുന്ന പ്രതിബന്ധങ്ങളുടെയും ഫലമായി ഷെയർ വിപണിയിൽ സംഭവിച്ച ആഘാതത്തെക്കുറിച്ചുള്ള പ്രസക്തമായ വിശദാംശങ്ങൾ ശരിയായി തിരിച്ചറിയുകയും സംക്ഷിപ്തമായി അവതരിപ്പിക്കുകയും ചെയ്തതെന്ന് സൂചിപ്പിക്കുന്നു.

#### ഉറവിട നോഡുകളുടെ വ്യാഖ്യാനം
ഉറവിട നോഡുകൾ അന്താരാഷ്ട്ര പ്രതിബന്ധങ്ങൾ കാരണം റഷ്യ നേരിട്ട സാമ്പത്തിക പ്രത്യാഘാതങ്ങളുടെ വിശദമായ കണക്കുകൾ നൽകുന്നു. എഴുത്തിൽ റഷ്യയുടെ ഷെയർ വിപണി അതിന്റെ മൂല്യത്തിന്റെ 40% നഷ്ടമായി എന്നതും വ്യാപാരം നിർത്തിവെക്കപ്പെട്ടതും ویژهമായി ഉയർത്തിപ്പറയുന്നു. കൂടാതെ, റൂബിൾയുടെ മൂല്യം ഇടിഞ്ഞത് പോലുള്ള മറ്റ് സാമ്പത്തിക പ്രതികാരങ്ങളും റഷ്യയുടെ സമ്പദ്‌വ്യവസ്ഥയുടെ വ്യാപകമായ ഒറ്റപ്പെടുത്തലും ഇതിൽ പരാമർശിച്ചിരിക്കുന്നു. LLM പ്രതികരണം ഇവയിൽ നിന്നുള്ള നിർണായക പോയിന്റുകൾ ഫലപ്രദമായി സംക്ഷിപ്തമാക്കി, ചോദനപ്രകാരം ഷെയർ വിപണിയിലുണ്ടായ ആഘാതത്തെ കേന്ദ്രീകരിച്ച് പ്രതിപാദിച്ചു.


ഇപ്പോൾ, ഹൈബ്രിഡ് സേർച്ച് നല്ല അടിസ്ഥാനമുള്ള മറുപടി നൽകാത്ത ഒരു ചോദ്യം നോക്കാം:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ഹൈബ്രിഡ് സെർച്ച്: LLM പ്രതികരണ വിശ്ലേഷണം
ഹൈബ്രിഡ് സെർച്ച് ഉദാഹരണത്തിലെ LLM പ്രതികരണം നൽകിയ കോൺടെക്സ് റഷ്യയുടെ യുക്രെയ്‌നിൽ നടത്തിയ ആക്രമণের കൃത്യമായ തീയതി വ്യക്തമാക്കിയില്ലെന്ന് സൂചിപ്പിക്കുന്നു. ഈ പ്രതികരണം LLM സോഴ്സ് ഡോക്യുമെന്റുകളിൽ ലഭ്യമായ വിവരങ്ങൾ ഉപയോഗിച്ചുകൊണ്ടിരിക്കുകയാണ് എന്നా൯ സൂചിപ്പിക്കുന്നു, എന്നാൽ ടെക്സ്റ്റിൽ കൃത്യമായ വിശദാംശങ്ങൾ ഇല്ലെന്നതു് അത് അംഗീകരിക്കുന്നു.

പ്രതികരണം റഷ്യയുടെ ആക്രമണവുമായി ബന്ധപ്പെട്ട घटनങ്ങൾ കോൺടെക്സ് പരാമർശിക്കുന്നുവെന്ന് തിരിച്ചറിയുന്നതിൽ യഥാർത്ഥമാണ്, പക്ഷേ പ്രത്യേകമായ ആക്രമণ തീയതി വ്യക്തമാക്കാൻ അത് കഴിയുന്നില്ല. ഇതിലൂടെ നൽകിയിരിക്കുന്ന വിവരങ്ങൾ മനസ്സിലാക്കാനുള്ള LLMയുടെ കഴിവും ഉള്ളടക്കത്തിലെ ഒഴിവുകൾ തിരിച്ചറിയാനുള്ള ജാഗ്രതയും കാണിക്കുന്നു. വിവരങ്ങൾ അപൂർണ്ണമായപ്പോൾ LLM ഉപയോക്താവിനെ കൃത്യമായ തീയതിക്ക് പുറമേന്ന സ്രോതസ്സുകൾ അല്ലെങ്കിൽ ചരിത്ര രേഖകൾ പരിശോധിക്കാൻ പ്രേരിപ്പിക്കുന്നു, ഇത് ഒരു തരം സൂക്ഷ്മത പ്രകടിപ്പിക്കുന്നു.

### സോഴ്‌സ് നോഡുകൾ വിശ്ലേഷണം
ഹൈബ്രിഡ് സെർച്ച് ഉദാഹരണത്തിലെ സോഴ്‌സ് നോഡുകൾ യുക്രെയ്‌നിൽ റഷ്യയുടെ നടപടി സംബന്ധിച്ച് യുഎസിന്റെ പ്രതികരണം ചർച്ച ചെയ്യുന്ന ഒരു പ്രസംഗത്തിൽ നിന്നുള്ള ഉദ്ധരണികൾ അടങ്ങിയവയാണ്. ഈ നോഡുകൾ ആക്രമണത്തിന് മറുപടിയായി യുഎസും അതിന്റെ സഖ്യരും സ്വീകരിച്ചിരിക്കുന്ന പ്രവർത്തനങ്ങളെക്കും വ്യാപകമായ ജിയോപൊളിറ്റിക്കൽ പ്രതിഭാവിനും ഊന്നൽ നല്കുന്നു, എന്നാൽ അവയിൽ നിശ്ചിത ആക്രമണ തീയതി പരാമർശിച്ചിട്ടില്ല. ഇത് കോൺടെക്സ് കൃത്യമായ തീയതിവിവരങ്ങൾക്കുറവാണെന്ന് ശരിയായി തിരിച്ചറിയുന്ന LLMയുടെ പ്രതികരണത്തോടൊപ്പം പൊരുത്തപ്പെടുന്നു.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: LLM പ്രതികരണ വിശകലനം
In the Hybrid w/Reranking example, the LLM response provides additional context by noting that the event occurred six days before the speech was given. This indicates that the LLM is able to infer the invasion date based on the timing of the speech, even though it still requires knowing the exact date of the speech for precision.

This response demonstrates an improved ability to use context clues to provide a more informative answer. It highlights the advantage of reranking, where the LLM can access and prioritize more relevant information to give a closer approximation of the desired detail (i.e., the invasion date).

### Source Nodes Analysis
The source nodes in this example include references to the timing of Russia's invasion, specifically mentioning that it occurred six days before the speech. While the exact date is still not explicitly stated, the nodes provide temporal context that allows the LLM to give a more nuanced response. The inclusion of this detail showcases how reranking can improve the LLM's ability to extract and infer information from the provided context, resulting in a more accurate and informative response.


**കുറിപ്പ്:**
ഈ നോട്ട്‌ബുക്കിൽ, നാം NVIDIA API Catalog-ൽ നിന്നുള്ള NVIDIA NIM മൈക്രോസർവിസുകൾ použitorzyc. 
മുകളിൽ പറഞ്ഞ APIകൾ, `NVIDIA (llms)`, `NVIDIAEmbedding`, കൂടാതെ [Azure AI Search സെമാന്റിക് ഹൈബ്രിഡ് റിട്രീവൽ (ബിൽറ്റ്-ഇൻ റീ-റാങ്കിംഗ്)](https://learn.microsoft.com/azure/search/semantic-search-overview). കുറിപ്പ്: മേൽപ്പറഞ്ഞ APIകൾ സ്വയം-ഹോസ്റ്റ് ചെയ്ത മൈക്രോസർവീസുകൾക്കും പിന്തുണ നൽകാം. 

**ഉദാഹരണം:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
അറിയിപ്പ്:
ഈ രേഖ AI തർജ്ജമാ സേവനമായ Co-op Translator (https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് തർജ്മ ചെയ്തതാണ്. ഞങ്ങൾ കൃത്യതയ്ക്ക് ശ്രമിച്ചാലും യന്ത്രം ഉപയോഗിച്ച തർജ്മകളിൽ പിശകുകൾ അല്ലെങ്കിൽ അപൂർവതകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയെ പ്രാമാണിക സ്രോതസ്സായി പരിഗണിക്കുക. നിർണായകമായ വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യ തർജ്ജമ ശുപാർശിക്കുന്നു. ഈ തർജ്മയുടെ ഉപയോഗം മൂലം ഉണ്ടായേക്കാവുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കും തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കും ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
